In [3]:
%pylab inline
import numpy as np
from sklearn.feature_selection import *

Populating the interactive namespace from numpy and matplotlib


# Removing features with low variance

가장 쉽게 접근할수 있으며, 모델에 영향력을 갖고 있지 않는 데이터들을 빠르게 제거할 수 있는 방법입니다.<br>
VarianceThreshold는 지속적으로 zero-variance features 그리고 지속적으로 동일한 값을 내놓은 features들을 제거합니다.

boolean 데이터에 대해서 feature selection을 하려고 합니다. <br>
Success(True) 또는 Failure(False)같이 2개로만 나눠지는 경우 **Bernoulli Distribution**을 따릅니다. <br>

성공(True)값은 $ p $ 로 나타내며, 실패(False)는 $ 1 - p $ 로 정의를 합니다.<br>
Bernoulli Random Variable의 Variance구하는 공식은 다음과 같습니다.

$$ Var[X] = p(1-p) $$

아래의 예제에서는 80%이상이 0 또는 1이 지속되는 데이터를 삭제를 합니다.

In [69]:
def var(data):
    return round(np.var(data), 2)

# List of Variances
data = np.array([[0, 0, 1], 
                 [0, 1, 0], 
                 [1, 0, 0], 
                 [0, 1, 1], 
                 [0, 1, 0], 
                 [0, 1, 1]])
print('[0]:{0} \n[1]:{1} \n[2]:{2}'.format(var(data[:, 0]), var(data[:, 1]), var(data[:, 2])))

valsel = VarianceThreshold(threshold=0.8 * (1-0.8))  # 0.8 * (1-0.8) = 0.15999
valsel.fit_transform(data)

[0]:0.14 
[1]:0.22 
[2]:0.25


array([[0, 1],
       [1, 0],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 1]])